## Day 10: Factory
### Part 1
Just across the hall, you find a large factory. Fortunately, the Elves here have plenty of time to decorate. Unfortunately, it's because the factory machines are all offline, and none of the Elves can figure out the initialization procedure.

The Elves do have the manual for the machines, but the section detailing the initialization procedure was eaten by a Shiba Inu. All that remains of the manual are some indicator light diagrams, button wiring schematics, and joltage requirements for each machine.

**_For example_**:

- [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
- [...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
- [.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}

The manual describes one machine per line. Each line contains a single indicator light diagram in [square brackets], one or more button wiring schematics in (parentheses), and joltage requirements in {curly braces}.

To start a machine, its indicator lights must match those shown in the diagram, where . means off and # means on. The machine has the number of indicator lights shown, but its indicator lights are all initially off.

So, an indicator light diagram like [.##.] means that the machine has four indicator lights which are initially off and that the goal is to simultaneously configure the first light to be off, the second light to be on, the third to be on, and the fourth to be off.

You can toggle the state of indicator lights by pushing any of the listed buttons. Each button lists which indicator lights it toggles, where 0 means the first light, 1 means the second light, and so on. When you push a button, each listed indicator light either turns on (if it was off) or turns off (if it was on). You have to push each button an integer number of times; there's no such thing as "0.5 presses" (nor can you push a button a negative number of times).

So, a button wiring schematic like (0,3,4) means that each time you push that button, the first, fourth, and fifth indicator lights would all toggle between on and off. If the indicator lights were [#.....], pushing the button would change them to be [...##.] instead.

Because none of the machines are running, the joltage requirements are irrelevant and can be safely ignored.

You can push each button as many times as you like. However, to save on time, you will need to determine the fewest total presses required to correctly configure all indicator lights for all machines in your list.

**_There are a few ways to correctly configure the first machine_**:

- [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}

- You could press the first three buttons once each, a total of 3 button presses.
- You could press (1,3) once, (2,3) once, and (0,1) twice, a total of 4 button presses.
- You could press all of the buttons except (1,3) once each, a total of 5 button presses.

However, the fewest button presses required is 2. One way to do this is by pressing the last two buttons ((0,2) and (0,1)) once each.

**_The second machine can be configured with as few as 3 button presses_**:

- [...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}

One way to achieve this is by pressing the last three buttons ((0,4), (0,1,2), and (1,2,3,4)) once each.

**_The third machine has a total of six indicator lights that need to be configured correctly_**:

- [.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}

The fewest presses required to correctly configure it is 2; one way to do this is by pressing buttons (0,3,4) and (0,1,2,4,5) once each.

So, the fewest button presses required to correctly configure the indicator lights on all of the machines is 2 + 3 + 2 = 7.

Analyze each machine's indicator light diagram and button wiring schematics. What is the fewest button presses required to correctly configure the indicator lights on all of the machines?

In [69]:
import itertools

def get_button_presses(machines):
    def str_to_set(s):
        s = s.strip("()")
        if not s:
            return set()
        return {int(x) for x in s.split(",")}

    def flatten_button_press(tuples):
        full_xor_set = set()
        for s in tuples: 
            full_xor_set = full_xor_set ^ s

        return full_xor_set

    def find_char_in_string(char, string): 
            return {i for i, x in enumerate(string) if x == char}

    machine_manual = dict()
    for i, machine in enumerate(machines): 
        machine_config = machine.split(' ')

        machine_manual[i] = {
            'indicator_light': machine_config[0], 
            'button_wiring': [str_to_set(x) for x in machine_config[1:-1]], 
            'joltage_req': machine_config[-1]
        }

    results = []
    for machine_id in machine_manual.keys(): 
        machine = machine_manual[machine_id]

        indicator_light = machine['indicator_light']
        button_wiring = machine['button_wiring']

        light_locations = set(find_char_in_string('#', indicator_light[1:-1]))
        # print(light_locations)

        # max len of presses is total length of buttons (on and off will always alternate), ordering doesn't matter
        combos = {i: [] for i in range(1, len(button_wiring))}
        for i in range(1, len(button_wiring)): 
            all_combos_i = list(itertools.combinations(button_wiring, i))
            # print(i, all_combos_i, '\n')
            for combo in all_combos_i: 
                combos[i].append(flatten_button_press(combo))
        
        found = False

        for i, potential_combos in combos.items(): 
            for combo in potential_combos:
                if light_locations == combo: 
                    found = True
                    results.append(i)
                    break

            if found: 
                break

    return sum(results)


In [70]:
with open('test_input.txt', 'r') as f:
    test_machines = []
    for line in f: 
        test_machines.append(line.strip())

get_button_presses(test_machines)

7

In [71]:
with open('input.txt', 'r') as f:
    machines = []
    for line in f: 
        machines.append(line.strip())

get_button_presses(machines)

385

### Part 2
All of the machines are starting to come online! Now, it's time to worry about the joltage requirements.

Each machine needs to be configured to exactly the specified joltage levels to function properly. Below the buttons on each machine is a big lever that you can use to switch the buttons from configuring the indicator lights to increasing the joltage levels. (Ignore the indicator light diagrams.)

The machines each have a set of numeric counters tracking its joltage levels, one counter per joltage requirement. The counters are all initially set to zero.

So, joltage requirements like {3,5,4,7} mean that the machine has four counters which are initially 0 and that the goal is to simultaneously configure the first counter to be 3, the second counter to be 5, the third to be 4, and the fourth to be 7.

**_The button wiring schematics are still relevant_**: in this new joltage configuration mode, each button now indicates which counters it affects, where 0 means the first counter, 1 means the second counter, and so on. When you push a button, each listed counter is increased by 1.

So, a button wiring schematic like (1,3) means that each time you push that button, the second and fourth counters would each increase by 1. If the current joltage levels were {0,1,2,3}, pushing the button would change them to be {0,2,2,4}.

You can push each button as many times as you like. However, your finger is getting sore from all the button pushing, and so you will need to determine the fewest total presses required to correctly configure each machine's joltage level counters to match the specified joltage requirements.

**_Consider again the example from before_**:

- [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
- [...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
- [.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}

.

- Configuring the first machine's counters requires a minimum of 10 button presses. One way to do this is by pressing (3) once, (1,3) three times, (2,3) three times, (0,2) once, and (0,1) twice.

- Configuring the second machine's counters requires a minimum of 12 button presses. One way to do this is by pressing (0,2,3,4) twice, (2,3) five times, and (0,1,2) five times.

- Configuring the third machine's counters requires a minimum of 11 button presses. One way to do this is by pressing (0,1,2,3,4) five times, (0,1,2,4,5) five times, and (1,2) once.

So, the fewest button presses required to correctly configure the joltage level counters on all of the machines is 10 + 12 + 11 = 33.

Analyze each machine's joltage requirements and button wiring schematics. What is the fewest button presses required to correctly configure the joltage level counters on all of the machines?

In [87]:
import ast
import numpy as np

def get_button_presses_joltage(machines):
    def str_to_set(s):
        s = s.strip("()")
        if not s:
            return set()
        return {int(x) for x in s.split(",")}

    def flatten_button_press(tuples):
        full_xor_set = set()
        for s in tuples: 
            full_xor_set = full_xor_set ^ s

        return full_xor_set

    def build_matrix_col(indices, joltage_req): 
        matrix_col = [0] * len(joltage_req)
        for i in indices: 
            matrix_col[i] = 1
        
        return matrix_col

    machine_manual = dict()
    for i, machine in enumerate(machines): 
        machine_config = machine.split(' ')

        machine_manual[i] = {
            'indicator_light': machine_config[0], 
            'button_wiring': [str_to_set(x) for x in machine_config[1:-1]], 
            'joltage_req': machine_config[-1]
        }

    results = []
    for machine_id in machine_manual.keys(): 
        machine = machine_manual[machine_id]

        button_wiring = machine['button_wiring']
        joltage_req = machine['joltage_req']

        joltage_req = ast.literal_eval(joltage_req.replace('{', '[').replace('}', ']'))

        matrix_cols = []
        for button in button_wiring: 
            matrix_cols.append(build_matrix_col(button, joltage_req))
        matrix_rows = [list(row) for row in zip(*matrix_cols)]

        for row in matrix_rows: 
            print(row)
        
        break


get_button_presses_joltage(test_machines)

[0, 0, 0, 0, 1, 1]
[0, 1, 0, 0, 0, 1]
[0, 0, 1, 1, 1, 0]
[1, 1, 0, 1, 0, 0]
